##installing libraries


In [1]:
!pip install transformers
!pip install sklearn
!pip install torch
!pip install wandb -qqq


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## importing the libraries

In [2]:
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from torch import nn
from transformers import BertModel
import math
from torch.optim import Adam
from tqdm import tqdm
import wandb

## login and setting weights and bisas

In [3]:
wandb.login()

wandb: Currently logged in as: amit1541541. Use `wandb login --relogin` to force relogin


True

In [4]:
def wand_set(lr,epochs):
    wandb.init(
          # Set the project where this run will be logged
          project="bert", 
          # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
          name=f"experiment_1", 
          # Track hyperparameters and run metadata
          config={
          "learning_rate":lr,
          "architecture": "Transformer",
          "dataset": "Peer Read",
          "epochs": epochs,
          })

## setting data set

we normelized the Recomndation score between 0-1

In [5]:
df = pd.read_csv("/content/data.csv")
df = df.dropna(subset=['comments','RECOMMENDATION'])
df['RECOMMENDATION'] = df['RECOMMENDATION'].astype('int')
df['RECOMMENDATION'] = df['RECOMMENDATION']-1
df['output'] = df['RECOMMENDATION'] / 10
df['output'] = df['output'].astype('float32')

In [6]:
def split_data(df):
    np.random.seed(112)
    df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                         [int(.8*len(df)), int(.9*len(df))])
    return df_train, df_val, df_test

In [7]:
df_train, df_val, df_test = split_data(df)


##setting bert tokenizer

In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

## torch Dataset class and model architecture

In [9]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [label for label in df['output']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['comments']]
    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

the model architecture we chose is a bert follwing by one hiden layer with 10 nureouns to one output neuron with Relu activation

In [10]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 1)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)
        return final_layer

## setting train and validation dataloaders to the teacher model


In [11]:
train, val = Dataset(df_train), Dataset(df_val)

train_dataloader = torch.utils.data.DataLoader(train, batch_size=24, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val, batch_size=24)

##train function for the models

we used MSE for the loss function and Adam optimizer

In [12]:
def train(model, train_dataloader, val_dataloader, learning_rate, epochs,batch_size=24):
    wand_set(learning_rate,epochs)
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.MSELoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs): 
        total_loss_train = 0
        iteration = 1
        for train_input, train_label in tqdm(train_dataloader):
            train_label = train_label.to(device)
            mask = train_input['attention_mask'].to(device)
            input_id = train_input['input_ids'].squeeze(1).to(device)

            output = model(input_id, mask)
            output = torch.flatten(output).to(torch.float32)
            train_label = train_label.to(torch.float32)
            batch_loss = criterion(output, train_label)
            total_loss_train += batch_loss.item()
            model.zero_grad()
            batch_loss.backward()
            optimizer.step()
            if iteration%10 == 0:
                wandb.log({"Train Loss": (total_loss_train/iteration)/batch_size})
            iteration += 1
            
        total_loss_val = 0
        with torch.no_grad():
            iteration = 1
            for val_input, val_label in val_dataloader:
                val_label = val_label.to(device)
                mask = val_input['attention_mask'].to(device)
                input_id = val_input['input_ids'].squeeze(1).to(device)
                output = model(input_id, mask)
                output = torch.flatten(output).to(torch.float32)
                val_label = val_label.to(torch.float32)
                batch_loss = criterion(output, val_label)
                total_loss_val += batch_loss.item()
                if iteration%10 == 0:
                    wandb.log({"Val loss": (total_loss_train/iteration)/batch_size})
                iteration+=1
                
        Train_loss = total_loss_train / len(df_train)
        Val_loss = total_loss_val / len(df_val)
        print(
            f'Epochs: {epoch_num + 1} | Train Loss: {Train_loss} \
            | Val Loss: {Val_loss}')
        wandb.log({"epoch_train loss": Train_loss, "epoch_val loss": Val_loss})
    wandb.finish()
            

##training the teacher model

In [13]:
teacher_model = BertClassifier()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
EPOCHS = 25
LR = 1e-6
              
train(teacher_model, train_dataloader, val_dataloader, LR, EPOCHS)

100%|██████████| 99/99 [03:19<00:00,  2.02s/it]


Epochs: 1 | Train Loss: 0.006445492858889745             | Val Loss: 0.006365585140883923


100%|██████████| 99/99 [03:25<00:00,  2.08s/it]


Epochs: 2 | Train Loss: 0.004674546237776526             | Val Loss: 0.004040326923131943


100%|██████████| 99/99 [03:27<00:00,  2.09s/it]


...

100%|██████████| 99/99 [03:26<00:00,  2.09s/it]


Epochs: 24 | Train Loss: 0.00041536905849273687             | Val Loss: 0.0008179282649999132


100%|██████████| 99/99 [03:26<00:00,  2.09s/it]


Epochs: 25 | Train Loss: 0.0003828140201800315             | Val Loss: 0.0008652538086312848


Train Loss,██▆▅▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Val loss,█▆▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch_train loss,█▆▄▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁
epoch_val loss,█▅▃▃▃▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁
Train Loss,0.00039
Val loss,0.00377
epoch_train loss,0.00038
epoch_val loss,0.00087


## Setting the data for the student model

In [15]:
def get_pseudo_labels(model, dataloader):
    outputs = []
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    
    if use_cuda:
        model = model.cuda()
        with torch.no_grad():
            for test_input, test_label in tqdm(dataloader):
                mask = test_input['attention_mask'].to(device)
                input_id = test_input['input_ids'].squeeze(1).to(device)
                output = model(input_id, mask)
                for i in range(30):
                    try:
                        outputs.append(output[i])
                    except:
                        pass
    outputs = [float(i) for i in outputs]
    return outputs


In [16]:
df_psudo = pd.read_csv("/content/data.csv")
df_psudo = df_psudo.dropna(subset=['comments'])
df_psudo['RECOMMENDATION'] = df_psudo['RECOMMENDATION']-1
df_psudo['output'] = df_psudo['RECOMMENDATION'] / 10
df_psudo['output'] = df_psudo['output'].astype('float32')

In [17]:
data = Dataset(df_psudo)
dataloader = torch.utils.data.DataLoader(data, batch_size=24, shuffle=True)
outputs = get_pseudo_labels(teacher_model, dataloader)
df_psudo['psudo'] =  outputs

100%|██████████| 242/242 [03:12<00:00,  1.26it/s]


lets see the rounded distance count from the predicted score of the teacher model to the real score 

In [18]:
df_psudo['distance'] = round(abs(df_psudo['psudo'] - df_psudo['output']),1)
df_psudo.groupby('distance').count()

,conference,Date,title,id,IMPACT,SUBSTANCE,APPROPRIATENESS,MEANINGFUL_COMPARISON,SOUNDNESS_CORRECTNESS,ORIGINALITY,CLARITY,REVIEWER_CONFIDENCE,accepted,RECOMMENDATION,comments,output,psudo
distance,,,,,,,,,,,,,,,,,
0.0,504,504,504,504,59,60,48,51,66,80,77,501,460,504,504,504,504
0.1,918,918,918,918,133,129,88,105,147,158,144,909,837,918,918,918,918
0.2,660,660,660,660,89,87,64,77,102,127,105,655,600,660,660,660,660
0.3,496,496,496,496,103,94,78,86,113,119,118,492,426,496,496,496,496
0.4,235,235,235,235,49,49,39,46,55,59,63,233,200,235,235,235,235
0.5,105,105,105,105,29,25,20,20,36,37,33,104,88,105,105,105,105
0.6,26,26,26,26,9,7,5,5,8,12,7,26,21,26,26,26,26
0.7,9,9,9,9,2,2,1,2,3,2,2,9,8,9,9,9,9
0.8,2,2,2,2,1,1,1,2,2,1,1,2,1,2,2,2,2


setting the labels in the dataset for the student model

In [19]:
r = []
output_list = list(df_psudo['output'])
psudo = list(df_psudo['psudo'])
for i in range(len(output_list)):
    if math.isnan(output_list[i]):
        r.append(psudo[i])
    else:
        r.append(output_list[i])

df_psudo['output'] = r
df_psudo

,conference,Date,title,id,IMPACT,SUBSTANCE,APPROPRIATENESS,MEANINGFUL_COMPARISON,SOUNDNESS_CORRECTNESS,ORIGINALITY,CLARITY,REVIEWER_CONFIDENCE,accepted,RECOMMENDATION,comments,output,psudo,distance
2638,iclr_2017,2016,Boosted Residual Networks,683.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,The authors mention that they are not aiming t...,0.282676,0.282676,NaN
2639,iclr_2017,2016,Boosted Residual Networks,683.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,All three reviewers point to significant defic...,0.272613,0.272613,NaN
2640,iclr_2017,2016,Boosted Residual Networks,683.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,False,2.0,This paper proposes a boosting based ensemble ...,0.200000,0.360889,0.2
2641,iclr_2017,2016,Boosted Residual Networks,683.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,False,3.0,The authors mention that they are not aiming t...,0.300000,0.327292,0.0
2642,iclr_2017,2016,Boosted Residual Networks,683.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,False,2.0,The paper under consideration proposes a set o...,0.200000,0.216101,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15265,conll_2016,2016,Parsing for Universal Dependencies without tra...,151.0,4.0,5.0,5.0,5.0,4.0,4.0,5.0,4.0,NaN,3.0,The authors proposed an unsupervised algorithm...,0.300000,0.436735,0.1
15266,conll_2016,2016,Parsing for Universal Dependencies without tra...,151.0,3.0,3.0,5.0,3.0,4.0,2.0,4.0,4.0,NaN,2.0,This paper presents a way to parse trees (name...,0.200000,0.429498,0.2
15267,conll_2016,2016,Event Linking with Sentential Features from Co...,13.0,3.0,4.0,5.0,5.0,4.0,3.0,4.0,4.0,NaN,3.0,This paper models event linking using CNNs. Gi...,0.300000,0.394215,0.1
15268,conll_2016,2016,Event Linking with Sentential Features from Co...,13.0,2.0,4.0,5.0,3.0,3.0,3.0,5.0,4.0,NaN,2.0,This paper presents a model for the task of ev...,0.200000,0.544733,0.3


In [20]:
df_train, df_val = train_test_split(df_psudo, test_size=0.2)

t, v = Dataset(df_train), Dataset(df_val)

train_dataloader = torch.utils.data.DataLoader(t, batch_size=24, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(v, batch_size=24)

##training the student model

In [21]:
student_model = BertClassifier()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [22]:
EPOCHS = 25
LR = 1e-6
              
train(student_model, train_dataloader, val_dataloader, LR, EPOCHS,24)

100%|██████████| 194/194 [06:38<00:00,  2.05s/it]


Epochs: 1 | Train Loss: 0.005769004204333368             | Val Loss: 0.005035915746000306


100%|██████████| 194/194 [06:39<00:00,  2.06s/it]


Epochs: 2 | Train Loss: 0.004309599997444418             | Val Loss: 0.0038049779703904843


100%|██████████| 194/194 [06:40<00:00,  2.07s/it]


...

100%|██████████| 194/194 [06:41<00:00,  2.07s/it]


Epochs: 24 | Train Loss: 0.0008644422502371387             | Val Loss: 0.001098830167782204


100%|██████████| 194/194 [06:41<00:00,  2.07s/it]


Epochs: 25 | Train Loss: 0.0008196056928484025             | Val Loss: 0.0011355794050955568


Train Loss,█▇▆▅▄▃▃▂▂▂▂▂▂▂▁▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Val loss,█▃▃▂▂▃▁▂▃▁▂▁▁▂▁▁▂▁▂▂▁▁▁▁▂▁▁▂▁▁▂▁▁▁▁▂▁▁▂▁
epoch_train loss,█▆▄▃▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
epoch_val loss,█▆▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Loss,0.00082
Val loss,0.00396
epoch_train loss,0.00082
epoch_val loss,0.00114


testing the student and teacher models

In [23]:
test_dataloader = torch.utils.data.DataLoader(Dataset(df_test), batch_size=24, shuffle=True)

In [24]:
student_labels = get_pseudo_labels(student_model, test_dataloader)
teacher_labels = get_pseudo_labels(teacher_model, test_dataloader)
df_test['student_labels'] = student_labels
df_test['teacher_labels'] = teacher_labels
df_test['mse_teacher'] = (df_test['teacher_labels'] - df_test['output'])**2
df_test['mse_student'] = (df_test['student_labels'] - df_test['output'])**2

100%|██████████| 13/13 [00:09<00:00,  1.34it/s]


In [25]:
mse_student = sum(df_test['mse_student']) / len(df_test['mse_student'])
mse_teacher = sum(df_test['mse_teacher']) / len(df_test['mse_teacher'])
print("student model mse:" + str(mse_student))
print("teacher model mse:" + str(mse_teacher))

student model mse:0.04407974721861084
teacher model mse:0.05504566664891854
